## <div style="text-align: right">   第13章 </div>

##  <div style="text-align: right"> 友好的进程</div>
___

我们暂时把代码搁在一边，来讨论一个更高层次的概念。

### 13.1 对CoW好点

正如我们在讲解进程衍生那章所提到的一样， _fork(2)_ 创建了一个和父进程一模一样的子进程。它包含了父进程在内存中的一切内容。

实实在在地复制所有的数据产生的系统开销不容小觑，因此现代的Unix系统采用了写时复制（copy-on-write，CoW)的方法来克服这个问题。

顾名思义，CoW将实际的内容复制操作推迟到了真正需要写入的时候。

所以说父进程和子进程实际上是在共享内存中的数据，知道它们其中的某一个需要对数据进行修改，届时才会进行内存复制，使得两个进程保持适当的隔离。

```python
import os

arr = [1, 2, 3]

if os.fork() == 0:
    # 此时子进程已经完成初始化
    # 借助CoW，子进程并不需要复制变量arr，因为它并没有修改任何共享变量，
    # 因此可以继续从和父进程同样的内存位置中进行读取。
```

```python
import os

arr = [1, 2, 3]

if os.fork() == 0:
        # 此时子进程已经完成初始化
        # 由于CoW，arr不会被复制
        arr.append(4)
        # 上面的代码修改了数组，因此在进行修改之前需要为子进程创建一个该列表的副本。
        # 父进程中的这个列表不会收到影响。
```

在使用 _fork(2)_ 时，这可是个了不得的优势，因为它节省了资源。由于不需要对父进程的内存内容进行即刻复制，这意味着 _fork(2)_ 的执行速度很快。也同样意味着子进程只获得了它所需要的那部分数据的副本，其余的部分仍然可以共享。

> 对于python中的copy-to-write的更多讨论，可以参见stackoverflow上的[此条回答](https://stackoverflow.com/questions/14749897/python-multiprocessing-memory-usage)，